In [1]:
# 1. Upscale Image
import os
import glob
import cv2
import torch
from basicsr.archs.rrdbnet_arch import RRDBNet
from upscaler.utils.utils import RealESRGANer

F:\Python\Python310\lib\site-packages\torchvision\transforms\functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(


In [3]:
input_dir = 'upscaler/input' # input image or folder
output_dir = 'upscaler/output'

model_name = 'RealESRGAN_x4plus'
model_path = 'upscaler/weights/RealESRGAN_x4plus.pth'


dni_weight = None
tile = 256
tile_pad = 30
pre_pad = 0
half = False
gpu_id = 0
outscale = 2

In [4]:
if model_name == 'RealESRGAN_x4plus':
    model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=4)
    netscale = 4
    model = model.to(torch.device('cuda'))

In [5]:
upsampler = RealESRGANer(
    scale=netscale,
    model_path=model_path,
    dni_weight=dni_weight,
    model=model,
    tile=tile,
    tile_pad=tile_pad,
    pre_pad=pre_pad,
    half=half,
    gpu_id=gpu_id
)

In [6]:
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

if os.path.isfile(input_dir):
    paths = [input_dir]
else:
    paths = sorted(glob.glob(os.path.join(input_dir, '*')))

In [7]:
def upscaler(path):
    img_name, extension = os.path.splitext(os.path.basename(path))
    print(f'Upscaling {img_name}.{extension}')
    img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    print(f'Image shape: {img.shape}')
    output, _ = upsampler.enhance(img, outscale=outscale)    
    save_path = os.path.join(output_dir, f'{img_name}{extension}')
    cv2.imwrite(save_path, output)

Choose Image

In [9]:
upscaler("upscaler/input/06.png")

Upscaling 06..png
Image shape: (616, 1072, 3)
	Tile 1/15
	Tile 2/15
	Tile 3/15
	Tile 4/15
	Tile 5/15
	Tile 6/15
	Tile 7/15
	Tile 8/15
	Tile 9/15
	Tile 10/15
	Tile 11/15
	Tile 12/15
	Tile 13/15
	Tile 14/15
	Tile 15/15


All Images

In [ ]:
for idx, path in enumerate(paths):
    imgname, extension = os.path.splitext(os.path.basename(path))
    print('R-ESRGAN', idx, imgname)
    upscaler(path)

In [ ]:
# 2. Ultimate Upscale
import math
from PIL import Image, ImageDraw
import torch
import numpy as np

from diffusers import StableDiffusionInpaintPipeline

In [ ]:
tile_width = 512
tile_height = 512
padding = 32

In [ ]:
pipe = StableDiffusionInpaintPipeline.from_pretrained("runwayml/stable-diffusion-inpainting")

pipe = pipe.to("cuda", torch_dtype=torch.float16)

prompt = "a black background with a bunch of pink flowers on it and green leaves on the side of the image, fine foliage lace, Annabel Kidston, arts and crafts movement, a flemish Baroque"

In [ ]:
def init_draw(width, height):
        n_width = math.ceil((tile_width+padding) / 64) * 64
        n_height = math.ceil((tile_height+padding) / 64) * 64
        mask = Image.new("L", (width, height), "black")
        draw = ImageDraw.Draw(mask)
        return mask, draw, n_width, n_height

def calc_rectangle(xi, yi):
    x1 = xi * tile_width
    y1 = yi * tile_height
    x2 = xi * tile_width + tile_width
    y2 = yi * tile_height + tile_height

    return x1, y1, x2, y2

def linear_process(image, rows, cols):
    p_img_list = []
    print("Org Image Size: ", image.size)
    mask, draw, n_width, n_height = init_draw(image.width, image.height)
    for yi in range(rows):
        for xi in range(cols):
            draw.rectangle(calc_rectangle(xi, yi), fill="white")
            image_mask = mask
            image_mask.save(f"mask_{xi}_{yi}.png")
            coord = calc_rectangle(xi, yi)
            c_image= image.crop(coord)
            print("Crop Image Size: ", c_image.size)
            c_image.save(f"crop_{xi}_{yi}.png")
            processed = pipe(prompt=prompt, image=c_image, mask_image=image_mask, num_inference_steps=50, strength=0.3).images[0]
            processed.save(f"processed_{xi}_{yi}.png")
            draw.rectangle( calc_rectangle(xi, yi), fill="black")
            p_image = processed
            p_img_list.append(p_image)


    return p_img_list

In [ ]:
image = Image.open("upscaler/output/pattern25.png")
image = image.convert("RGB")
image_list = linear_process(image, 2, 2)

empty_image = Image.new("RGB", (image.width, image.height), "black")
for yi in range(2):
    for xi in range(2):
        coord = calc_rectangle(xi, yi)
        empty_image.paste(image_list[yi*2+xi], box=coord)

empty_image.save("output_image.png")